In [1]:
import pandas as pd
train_mobile = pd.read_csv('mobile.csv')
train_static = pd.read_csv('static.csv')

In [2]:
train_data_table = pd.concat([train_mobile, train_static])

In [3]:
train_data_table = train_data_table.reset_index(drop=True)

In [29]:
train_data_cell = []
count = []
flag = 0
for i in range(1,train_data_table.shape[0]):
    if train_data_table['time'][i] < train_data_table['time'][i-1]:
        count.append(i)
        if i > train_static.shape[0] and flag == 0:
            num_static = len(count)-1
            flag = 1
count = [0] + count + [train_data_table.shape[0]]
count.pop()
print(count)
for i in range(1, len(count)):
    train_data_cell.append(train_data_table.iloc[count[i-1]:count[i], :])
num_mobile = len(train_data_cell) - num_static
data_combine = train_data_cell

[0, 89483, 228914]


In [30]:
data_combine

[                      time    hmd    spd    tmp  pm2d5       lat        lon
 0      2018-05-10 06:19:12  38.07   0.22  16.60  58.67  39.10240  117.73839
 1      2018-05-10 06:19:15  38.50   0.26  16.60  63.33  39.10240  117.73839
 2      2018-05-10 06:19:18  38.77   0.35  16.63  64.00  39.10241  117.73839
 3      2018-05-10 06:19:21  39.03   0.31  16.70  63.33  39.10241  117.73839
 4      2018-05-10 06:19:24  39.33   0.13  16.70  63.00  39.10240  117.73840
 ...                    ...    ...    ...    ...    ...       ...        ...
 89478  2018-05-16 10:14:09  61.70  29.26  24.00  44.00  39.11575  117.74278
 89479  2018-05-16 10:14:12  61.70  30.96  24.00  44.00  39.11562  117.74252
 89480  2018-05-16 10:14:15  61.70  34.49  24.00  43.00  39.11547  117.74225
 89481  2018-05-16 10:14:18  61.70  31.87  24.00  42.67  39.11531  117.74200
 89482  2018-05-16 10:14:21  61.70  27.01    NaN    NaN       NaN        NaN
 
 [89483 rows x 7 columns],
                        time   hmd   spd   tmp 

In [31]:
import numpy as np

data_total_minute = [0]*len(data_combine)
for i in range(len(data_combine)):
    time = pd.to_datetime(data_combine[i]['time'])
    date = time.dt.date
    unique_date = date.unique()
    date_ = time.dt.strftime('%m/%d/%Y')
    dayName = time.dt.strftime('%A')
    hour_ = time.dt.strftime('%H')
    minute_ = time.dt.strftime('%M')
    dayNumber = time.dt.weekday
    data = data_combine[i].assign(date_=date_, dayName=dayName, hour_=hour_, minute_=minute_, dayNumber=dayNumber)
    data_gb_datemin = data.groupby(['date_', 'hour_', 'minute_']).mean()
    time_minutely = pd.to_datetime(date_ + ' ' + hour_ + ':' + minute_, format='%m/%d/%Y %H:%M')
    time_minutely = np.unique(time_minutely)
    hmd = data_gb_datemin['hmd']
    spd = data_gb_datemin['spd']
    tmp = data_gb_datemin['tmp']
    pm2d5 = data_gb_datemin['pm2d5']
    lat = data_gb_datemin['lat']
    lon = data_gb_datemin['lon']
    data_total_minute[i] = pd.DataFrame({'time':time_minutely, 'hmd':hmd, 'spd':spd, 'tmp':tmp, 'pm2d5':pm2d5, 'lat':lat, 'lon':lon})
    # data_total_minute[i] = data_total_minute[i].rename(columns={0: "time", 1:'hmd', 2:'spd', 3:'tmp', 4:'pm2d5', 5:'lat', 6:'lon'})

In [32]:
data_total_minute

[                                        time      hmd        spd       tmp  \
 date_      hour_ minute_                                                     
 05/10/2018 06    19      2018-05-10 06:19:00  40.0600   0.240625  16.78125   
                  20      2018-05-10 06:20:00  42.9520   0.191500  17.13350   
                  21      2018-05-10 06:21:00  44.4925   0.208500  17.41850   
                  22      2018-05-10 06:22:00  45.3150   0.176500  17.65050   
                  23      2018-05-10 06:23:00  45.8330   0.175500  17.84500   
 ...                                      ...      ...        ...       ...   
 05/16/2018 10    10      2018-05-16 10:10:00  61.5000  10.613500  24.00000   
                  11      2018-05-16 10:11:00  61.5785  24.066500  24.00000   
                  12      2018-05-16 10:12:00  61.6135  14.670500  24.00000   
                  13      2018-05-16 10:13:00  61.7015  21.952000  24.00000   
                  14      2018-05-16 10:14:00  61.70

In [33]:
scale = 1
scale_parameter = 0.5

In [34]:
num_static,num_mobile

(1, 1)

In [35]:
data_cleaned_static = []
for i in range(num_static):  # only static
    T = 60 * 24 * scale
    sw25 = [1/(2*T)] + [1/T] * (T - 1) + [1/(2*T)]
    trend = np.convolve(data_total_minute[i]['pm2d5'], sw25, 'same')

    # padding the trend
    trend_padded = np.pad(trend[int(T/2):-int(T/2)], (int(T/2), int(T/2)), "constant", constant_values=(trend[int(T/2)], trend[-int(T/2)]))

    T1 = len(data_total_minute[i]['pm2d5'])
    sidx = {}
    for ii in range(T):
        sidx[ii] = list(range(ii, T1, T))

    detrend_data = data_total_minute[i]['pm2d5'] - trend_padded  # detrend
    seasonal = [np.mean(detrend_data[sidx[ii]]) for ii in range(T)]  # generate seasonal trend
    nc = T1 // T
    rm = T1 % T
    seasonal = seasonal * nc + seasonal[:rm]
    sBar = np.mean(seasonal)
    seasonal = seasonal - sBar
    residual = data_total_minute[i]['pm2d5'] - trend_padded - seasonal
    pm2d5_clean = trend_padded + seasonal
    time = data_total_minute[i]['time']
    data_cleaned_static.append(pd.DataFrame({'time': time, 'pm2d5_clean': pm2d5_clean}))

In [36]:
# seasonal decomposition for mobile
data_cleaned_mobile = []
for i in range(num_static,num_static+num_mobile):  # only mobile
    T = 60 * 24 * scale
    sw25 = [1/(2*T)] + [1/T] * (T - 1) + [1/(2*T)]
    trend = np.convolve(data_total_minute[i]['pm2d5'], sw25, 'same')

    # padding the trend
    trend_padded = np.pad(trend[int(T/2):-int(T/2)], (int(T/2), int(T/2)), "constant", constant_values=(trend[int(T/2)], trend[-int(T/2)]))

    T1 = len(data_total_minute[i]['pm2d5'])
    sidx = {}
    for ii in range(T):
        sidx[ii] = list(range(ii, T1, T))

    detrend_data = data_total_minute[i]['pm2d5'] - trend_padded  # detrend
    seasonal = [np.mean(detrend_data[sidx[ii]]) for ii in range(T)]  # generate seasonal trend
    nc = T1 // T
    rm = T1 % T
    seasonal = seasonal * nc + seasonal[:rm]
    sBar = np.mean(seasonal)
    seasonal = seasonal - sBar
    residual = data_total_minute[i]['pm2d5'] - trend_padded - seasonal
    pm2d5_clean = trend_padded + seasonal
    time = data_total_minute[i]['time']
    data_cleaned_mobile.append(pd.DataFrame({'time': time, 'pm2d5_clean': pm2d5_clean}))

In [37]:
from scipy.signal import gaussian

def smooth_data(data, window_size):
    # Generate a 1D Gaussian kernel
    kernel = gaussian(window_size, std=window_size / 4)
    # Normalize the kernel
    kernel /= kernel.sum()
    # Convolve the data with the kernel
    smooth_data = np.convolve(data, kernel, 'same')
    return smooth_data
window_size = 60*24*scale_parameter

data_total_pro = data_total_minute
for i in range(num_static):
    data_total_pro[i]['pm2d5'] = data_cleaned_static[i]['pm2d5_clean']
    data_total_pro[i]['hmd'] = smooth_data(data_total_pro[i]['hmd'], window_size)
    data_total_pro[i]['spd'] = smooth_data(data_total_pro[i]['spd'], window_size)
    data_total_pro[i]['tmp'] = smooth_data(data_total_pro[i]['tmp'], window_size)

In [38]:
#parameter cleaning for mobile sensor
for i in range(num_mobile):
    data_total_pro[i+num_static]['pm2d5'] = data_cleaned_mobile[i]['pm2d5_clean']
    data_total_pro[i+num_static]['hmd'] = smooth_data(data_total_pro[i+num_static]['hmd'], window_size)
    data_total_pro[i+num_static]['spd'] = smooth_data(data_total_pro[i+num_static]['spd'], window_size)
    data_total_pro[i+num_static]['tmp'] = smooth_data(data_total_pro[i+num_static]['tmp'], window_size)

In [39]:
import pandas as pd

#putting data into one matrix and sort the data
data_onematrix = pd.concat(data_total_pro)
data_total_sorted = data_onematrix.sort_values(by='time')

#normalize the data, using range
timenum_train = pd.to_datetime(data_total_sorted['time']).astype(int) // 10**9
data_total_norm_para = data_total_sorted[['time', 'hmd', 'spd', 'tmp', 'lat', 'lon']]
# data_total_norm_para_withtest = pd.concat([data_total_norm_para, test_data])
# data_total_norm_para_withtest = (data_total_norm_para_withtest - data_total_norm_para_withtest.min()) / (data_total_norm_para_withtest.max() - data_total_norm_para_withtest.min())
data_total_norm_para = (data_total_norm_para - data_total_norm_para.min()) / (data_total_norm_para.max() - data_total_norm_para.min())

<ipython-input-39-4609a1f1d42c>:8: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  timenum_train = pd.to_datetime(data_total_sorted['time']).astype(int) // 10**9


In [44]:
x = data_total_norm_para
y = data_total_sorted['pm2d5']

In [41]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
kfold = KFold(n_splits=5)
model = RandomForestRegressor(random_state=42)
scoring = 'neg_mean_squared_error'
results = cross_val_score(model, x, y, cv=kfold, scoring=scoring)

#calculate and print the mean and standard deviation of the RMSE
rmse_scores = np.sqrt(-results)
print(f"RMSE scores: {rmse_scores}")

RMSE scores: [11.267708   17.47308293 22.07761688 10.28664306 10.43647964]


In [45]:
x = x.interpolate()
y = y.interpolate()

In [46]:
x = x.values
x.reshape(-1,6)
x = np.reshape(x, (-1, 1, 6))
y = y.values
y = np.reshape(y, (-1, 1, 1))

In [62]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.optimizers import SGD

x = tf.convert_to_tensor(x, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)

# define the model
model = tf.keras.Sequential()
model.add(LSTM(64, input_shape=(1, 6), activation='tanh'))
model.add(Dense(1))

# compile the model with Adam optimization and RMSE loss function
model.compile(optimizer=Adam(learning_rate=0.008), loss='mean_squared_error')

# fit the model to the data
history = model.fit(x, y, epochs=10, batch_size=32, validation_split=0.2)

# use cross-validation to evaluate the model
scores = model.evaluate(x, y, verbose=0)
print("RMSE:", np.sqrt(scores))

Epoch 1/10
288/288 [==============================] - 3s 4ms/step - loss: 861.0839 - val_loss: 242.7426
Epoch 2/10
288/288 [==============================] - 1s 3ms/step - loss: 193.4423 - val_loss: 221.9699
Epoch 3/10
288/288 [==============================] - 1s 2ms/step - loss: 185.3744 - val_loss: 183.3255
Epoch 4/10
288/288 [==============================] - 1s 2ms/step - loss: 177.7236 - val_loss: 154.0195
Epoch 5/10
288/288 [==============================] - 1s 3ms/step - loss: 171.3399 - val_loss: 130.7650
Epoch 6/10
288/288 [==============================] - 1s 2ms/step - loss: 163.8833 - val_loss: 126.4249
Epoch 7/10
288/288 [==============================] - 1s 2ms/step - loss: 144.9061 - val_loss: 104.2445
Epoch 8/10
288/288 [==============================] - 1s 4ms/step - loss: 124.0482 - val_loss: 75.0902
Epoch 9/10
288/288 [==============================] - 1s 4ms/step - loss: 103.2515 - val_loss: 70.8601
Epoch 10/10
288/288 [==============================] - 1s 3ms/step